In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from collections import Counter
import sqlite3
import seaborn
from ipywidgets import interact, Output
import ipywidgets as widgets

###### These are configurations to be used for all plots

In [72]:
 # will show up to 4
price_min = 5000
price_max = 500000  # this will cut off some offers, but will make charts more informative
meter_price_min = 50
meter_price_max = 4000
hist_resolution = 200  # all values will be divided in this many groups
area_min = 0
area_max = 1000

###### Connecting to database and loading all data into Pandas DataFrame

In [56]:
conn = sqlite3.connect('offers.db')
cursor = conn.cursor()

data = pd.read_sql("SELECT * FROM houses", conn, index_col="olx_id")
conn.close()

In [57]:
data['total_area'].replace('', np.nan, inplace=True)

In [58]:
data.head(2)

,price,total_area,rooms,floors_in_house,parsed_date,offer_title,offer_from,house_type,district,offer_added_date,text,offer_url,land_area
olx_id,,,,,,,,,,,,,
68835071,100000,400.0,6.0,3,2019-09-16,Продам дом,Частного лица,Дом,Индустриальный,2019-09-09,"Продам дом на ХТЗ.Участок 7 соток, 2-ва дома н...",https://www.olx.ua/obyavlenie/prodam-dom-ID4EP...,7.0
84257427,40000,120.0,7.0,1,2019-09-16,Продам отдельно стоящее здание,Частного лица,Дом,Московский,2019-09-10,Продам отдельно стоящее здание площадь 115 м2|...,https://www.olx.ua/obyavlenie/prodam-otdelno-s...,1.0


Data cleanup. Filtering out disctricts from outside the city. Removing offers with unreal values for price and area.

In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2084 entries, 68835071 to 612427278
Data columns (total 13 columns):
price               2084 non-null int64
total_area          2084 non-null float64
rooms               2084 non-null float64
floors_in_house     2084 non-null object
parsed_date         2084 non-null object
offer_title         2084 non-null object
offer_from          2084 non-null object
house_type          2084 non-null object
district            2084 non-null object
offer_added_date    2084 non-null object
text                2084 non-null object
offer_url           2084 non-null object
land_area           2084 non-null float64
dtypes: float64(3), int64(1), object(9)
memory usage: 227.9+ KB


In [60]:
data = data[(data['district'].isin(['Шевченковский', 'Киевский', 'Фрунзенский', 'Холодногорский', 'Московский', 'Червонозаводской', 'Октябрьский', 'Индустриальный ', 'Коминтерновский']))]
data['offer_added_date'] = data.offer_added_date.astype('datetime64')

In [61]:
price_square_meter = data.price / data.total_area
price_square_meter.head(2)

olx_id
68835071    250.000000
84257427    333.333333
dtype: float64

### Code for histogram

In [73]:
def hist_set_x_axis(value):
    if value == 'Price':
        x_axis = data.price
        range_min = price_min
        range_max = price_max
    if value == 'Price per square meter':
        x_axis = price_square_meter
        range_min = meter_price_min
        range_max = meter_price_max
    if value == 'Total area':
        x_axis = data.total_area
        range_min = area_min
        range_max = area_max   
    if value == 'Land area':
        x_axis = data.land_area
        range_min = 0
        range_max = 20
        
    return (x_axis, range_min, range_max)

In [63]:
hist_x_axis_dropdown = widgets.Dropdown(
    options=['Price', 'Price per square meter', 'Total area', 'Land area'], 
    value='Price')

In [64]:
def build_histogram(x_axis):
    a, range_min, range_max = hist_set_x_axis(x_axis)
    plt.figure(figsize=(20,5), dpi=300, facecolor='white')
    plt.hist(a, hist_resolution, range=(range_min, range_max))
    plt.show()

In [65]:
hist_out = Output()

In [66]:
def x_axis_observer(bunch):
    hist_out.clear_output()
    with hist_out:
        build_histogram(x_axis = bunch.new)

In [67]:
hist_x_axis_dropdown.observe(x_axis_observer, names='value')

In [74]:
display(hist_x_axis_dropdown)
display(hist_out)

Dropdown(options=('Price', 'Price per square meter', 'Total area', 'Land area'), value='Price')

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 6000x1500 with 1 Axes>', …

### Code for bar chart

In [95]:
def bar_set_x_axis(value):
    rotation = 0
    if value == 'District':
        counter = Counter(data['district'])
    if value == 'Number of rooms':
        counter = Counter(data['rooms'])
    if value == 'House type':
        counter = Counter(data['house_type'])
        rotation = 30
    if value == 'Offer from':
        counter = Counter(data['offer_from'])
    if value == 'Floors in a house':
        counter = Counter(data['floors_in_house'])
        
    labels = dict(counter.most_common()).keys()
    height = dict(counter.most_common()).values()
    x_axis = np.arange(len(labels))
    
    return (x_axis, height, labels, rotation)

In [89]:
bar_x_axis_dropdown = widgets.Dropdown(
    options=['District', 'Number of rooms', 'House type', 'Offer from', 'Floors in a house',], 
    value='District')

In [90]:
def build_bar_chart(x_axis):
    x_axis, height, labels, rotation = bar_set_x_axis(x_axis)
    # Plot histogram using matplotlib bar().
    plt.figure(figsize=(20,5), dpi=300, facecolor='white')
    width = 0.9
    plt.bar(x_axis, height, width, align='center')
    plt.xticks(x_axis, labels, rotation=rotation)
    plt.show()

In [91]:
bar_out = Output()

In [92]:
def bar_x_axis_observer(bunch):
    bar_out.clear_output()
    with bar_out:
        build_bar_chart(x_axis = bunch.new)

In [93]:
bar_x_axis_dropdown.observe(bar_x_axis_observer, names='value')

In [96]:
display(bar_x_axis_dropdown)
display(bar_out)

Dropdown(index=5, options=('District', 'Number of rooms', 'House type', 'Offer from', 'Floors in a house', 'Of…

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 6000x1500 with 1 Axes>', …